# Nov 10, 2024: Canonical Resting State Networks
Joanes Grandjean provided us with his ICA components. Each IC corresponds to a network, or a sub-network. 
He groups his ICs hierarchically into resting state networks.

In [1]:
import csv
import os
import sys
import numpy as np
import pandas as pd
import scipy as sp 
import dill as pickle 
from os.path import join as pjoin
from itertools import product
from tqdm import tqdm
import ants
from glob import glob
import graph_tool.all as gt

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

gt.seed_rng(args.SEED)
np.random.seed(args.SEED)

In [3]:
args.type = 'spatial'
args.roi_size = 225
args.maintain_symmetry = True
args.brain_div = 'whl'
args.num_rois = 162

PARC_DESC = (
    f'type-{args.type}'
    f'_size-{args.roi_size}'
    f'_symm-{args.maintain_symmetry}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
)

In [4]:
args.GRAPH_DEF = f'constructed'
args.GRAPH_METHOD = f'pearson-corr'
args.THRESHOLDING = f'positive'
args.EDGE_DEF = f'binary'
args.EDGE_DENSITY = 20
args.LAYER_DEF = f'individual'
args.DATA_UNIT = f'ses'

BASE_path = f'{os.environ["HOME"]}/mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
RSN_path = f'{BASE_path}/joanes_rsns'
ROI_path = f'{BASE_path}/roi_results_v2/{PARC_DESC}'

TS_path = f'{ROI_path}/runwise_timeseries'
ROI_RESULTS_path = (
    f'{ROI_path}'
    f'/graph-{args.GRAPH_DEF}/method-{args.GRAPH_METHOD}'
    f'/threshold-{args.THRESHOLDING}/edge-{args.EDGE_DEF}/density-{args.EDGE_DENSITY}'
    f'/layer-{args.LAYER_DEF}/unit-{args.DATA_UNIT}'
)
RSN_ROI_path = f'{ROI_path}/rsns'
os.system(f'mkdir -p {RSN_ROI_path}')
GRAPH_path = f'{ROI_RESULTS_path}/graphs'
os.system(f'mkdir -p {GRAPH_path}')
SBM_path = f'{ROI_RESULTS_path}/model-fits'
os.system(f'mkdir -p {SBM_path}')

0

In [5]:
parcels_img = ants.image_read(f'{PARCELS_path}/{PARC_DESC}_desc-parcels.nii.gz')
parcels = parcels_img.numpy()
roi_labels = np.loadtxt(f'{PARCELS_path}/{PARC_DESC}_desc-labels.txt')

In [6]:
cmask_img = ants.image_read(
    f'{BASE_path}/voxel/common_brain_mask.nii.gz'
)
cmask_img

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (58, 79, 45)
	 Spacing    : (0.2, 0.2, 0.2)
	 Origin     : (18.1, 2.7, -7.8)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

---

In [7]:
rimg = ants.image_read(
    f'{BASE_path}/gabe_symmetric_N162/Symmetric_N162_0.20_RAS.nii.gz'
)
rimg

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (60, 81, 46)
	 Spacing    : (0.2, 0.2, 0.2)
	 Origin     : (18.2, 2.8, -7.8)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [8]:
ics_img = ants.image_read(
    f'{RSN_path}/ABI_DR_zerbi15_100um.nii.gz'
)
ics_img

ANTsImage
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (114, 132, 80, 18)
	 Spacing    : (0.1, 0.1, 0.1, 1.0)
	 Origin     : (5.44, 6.31, -4.325, 0.0)
	 Direction  : [-1.  0.  0.  0.  0. -1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]

In [9]:
jimg = ants.image_read(
    f'{RSN_path}/ABI_template_100um.nii'
)
jimg

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (114, 132, 80)
	 Spacing    : (0.1, 0.1, 0.1)
	 Origin     : (5.44, 6.31, -4.325)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [10]:
nimg = ants.image_read(
    f'{PARCELS_path}/allen_template_RAS_tx.nii.gz'
)
nimg

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (118, 160, 90)
	 Spacing    : (0.1, 0.1, 0.1)
	 Origin     : (18.1, 2.7, -7.8)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [11]:
tx = ants.registration(
    fixed=nimg, 
    moving=jimg,
    type_of_transform='SyN',
    random_seed=100,
)

In [12]:
jimg_tx = ants.apply_transforms(
    fixed=nimg, 
    moving=jimg,
    transformlist=tx['fwdtransforms'],
)
jimg_tx.to_file(f'{RSN_path}/transformed_to_N162/joanes_template_RAS_tx.nii.gz')
jimg_tx

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (118, 160, 90)
	 Spacing    : (0.1, 0.1, 0.1)
	 Origin     : (18.1, 2.7, -7.8)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [13]:
def threshold(arr, thresh=6):
    mask = np.abs(arr) <= thresh
    arr[mask] = 0
    return arr

In [14]:
ics = ics_img.numpy()
ics_tx = []
for ic in range(ics.shape[-1]):
    ic_img = jimg.new_image_like(ics[:, :, :, ic])
    ic_img_tx = ants.apply_transforms(
        fixed=nimg,
        moving=ic_img,
        transformlist=tx['fwdtransforms'],
    )
    ic_img_tx_200 = ants.resample_image_to_target(
        image=ic_img_tx, 
        target=rimg,
    )
    ics_tx += [threshold(ic_img_tx_200.numpy())]
    ic_img_tx_200.to_file(f'{RSN_path}/transformed_to_N162/ics/ic-{ic}.nii.gz')

ics_tx = np.stack(ics_tx, axis=-1)
ics_tx.shape

(60, 81, 46, 18)

---

In [15]:
# create RSNs

In [16]:
def create_rsn_from_ics(rimg, ics_tx, ics, name):
    rsn_img = rimg.new_image_like(
        np.sum(ics_tx[:, :, :, ics], axis=-1)
    )
    rsn_img.to_file(f'{RSN_path}/transformed_to_N162/rsns/j-{name}.nii.gz')
    return rsn_img

def create_rsn_from_atlas(rimg, atlas_img, rois, name):
    atlas = atlas_img.numpy()
    mask = np.isin(atlas, rois)
    rsn = atlas * mask
    rsn_img = rimg.new_image_like(rsn)
    rsn_img.to_file(f'{RSN_path}/transformed_to_N162/rsns/j-{name}.nii.gz')
    return rsn_img

def create_parcels_rsn(rsn_img, name, roi_labels):
    rsn_img = ants.resample_image_to_target(
        image=rsn_img, 
        target=cmask_img,
    )

    rsn = (rsn_img.numpy() > 0) * (parcels)
    rsn_mask = (rsn > 0).astype(np.float32)
    rsn_rois = np.unique(rsn)[1:]
    
    # rsn rois vector
    np.savetxt(
        f'{RSN_ROI_path}/desc-j-{name}-rois.txt',
        np.in1d(roi_labels, rsn_rois).astype(int),
        fmt='%1d',
    )
    # rsn rois img, and mask img
    cmask_img.new_image_like(rsn).to_file(f'{RSN_ROI_path}/desc-j-{name}-rois.nii.gz')
    cmask_img.new_image_like(rsn_mask).to_file(f'{RSN_ROI_path}/desc-j-{name}-mask.nii.gz')
    return rsn, rsn_mask, rsn_rois

In [17]:
ics = [2, 3, 4, 5]
name = 'somatosensory'
rsn_img = create_rsn_from_ics(rimg, ics_tx, ics, name)
rsn, rsn_mask, rsn_rois = create_parcels_rsn(rsn_img, name, roi_labels)

ics = [1, 6, 7]
name = 'sensory'
rsn_img = create_rsn_from_ics(rimg, ics_tx, ics, name)
rsn, rsn_mask, rsn_rois = create_parcels_rsn(rsn_img, name, roi_labels)

ics = [0, 17]
name = 'olfactory'
rsn_img = create_rsn_from_ics(rimg, ics_tx, ics, name)
rsn, rsn_mask, rsn_rois = create_parcels_rsn(rsn_img, name, roi_labels)

ics = [9, 8, 10, 11, 15, 16]
name = 'limbic'
rsn_img = create_rsn_from_ics(rimg, ics_tx, ics, name)
rsn, rsn_mask, rsn_rois = create_parcels_rsn(rsn_img, name, roi_labels)

ics = [13, 12, 14]
name = 'basal_ganglia'
rsn_img = create_rsn_from_ics(rimg, ics_tx, ics, name)
rsn, rsn_mask, rsn_rois = create_parcels_rsn(rsn_img, name, roi_labels)

ics = [6]
name = 'visual'
rsn_img = create_rsn_from_ics(rimg, ics_tx, ics, name)
rsn, rsn_mask, rsn_rois = create_parcels_rsn(rsn_img, name, roi_labels)

---

In [18]:
atlas_img = ants.image_read(
    f'{RSN_path}/ABI_atlas_reduced_V2_100um.nii.gz',
)
atlas_img_tx = ants.apply_transforms(
    fixed=nimg,
    moving=atlas_img,
    transformlist=tx['fwdtransforms'],
    interpolator='genericLabel',
)
atlas_img_tx_200 = ants.resample_image_to_target(
    image=atlas_img_tx, 
    target=rimg,
    interp_type='genericLabel',
)
atlas_img_tx_200.to_file(f'{RSN_path}/transformed_to_N162/joanes_ABI_atlas_200_RAS_tx.nii.gz')
atlas_img_tx_200

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (60, 81, 46)
	 Spacing    : (0.2, 0.2, 0.2)
	 Origin     : (18.2, 2.8, -7.8)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [19]:
rois = [38, 39, 40, 41, 84, 85, 86, 87]
name = 'amygdala'
rsn_img = create_rsn_from_atlas(rimg, atlas_img_tx_200, rois, name)
rsn, rsn_mask, rsn_rois = create_parcels_rsn(rsn_img, name, roi_labels)

rois = [45, 91]
name = 'thalamus'
rsn_img = create_rsn_from_atlas(rimg, atlas_img_tx_200, rois, name)
rsn, rsn_mask, rsn_rois = create_parcels_rsn(rsn_img, name, roi_labels)

rois = [46, 92]
name = 'amygdala'
rsn_img = create_rsn_from_atlas(rimg, atlas_img_tx_200, rois, name)
rsn, rsn_mask, rsn_rois = create_parcels_rsn(rsn_img, name, roi_labels)

rois = [2, 3, 4, 5, 6, 7, 8, 9, 10, 19, 48, 49, 50, 51, 52, 53, 54, 55, 56, 65]
name = 'somatomotor'
rsn_img = create_rsn_from_atlas(rimg, atlas_img_tx_200, rois, name)
rsn, rsn_mask, rsn_rois = create_parcels_rsn(rsn_img, name, roi_labels)

rois = [1, 15, 16, 17, 18, 20, 22, 47, 62, 63, 61, 64, 66, 68]
name = 'default_mode'
rsn_img = create_rsn_from_atlas(rimg, atlas_img_tx_200, rois, name)
rsn, rsn_mask, rsn_rois = create_parcels_rsn(rsn_img, name, roi_labels)

rois = [25, 26, 42, 43, 44, 45, 46, 71, 72, 88, 89, 90, 91, 92]
name = 'subcortical'
rsn_img = create_rsn_from_atlas(rimg, atlas_img_tx_200, rois, name)
rsn, rsn_mask, rsn_rois = create_parcels_rsn(rsn_img, name, roi_labels)